In [1]:
import pandas as pd
import sqlite3
import os
import sys
from datetime import datetime

print("=== SciSciNet Preprocessing Notebook ===")
print(f"Python version: {sys.version}")
print(f"Pandas version: {pd.__version__}\n")

# Load validation config
try:
    from validation_config import (
        VT_AFFILIATION_IDS, CS_FIELD_IDS, DATA_DIR,
        AFFILIATION_ID_COL, AFFILIATION_NAME_COL,
        FIELD_ID_COL, FIELD_NAME_COL,
        PAPER_ID_COL, AUTHOR_ID_COL, PAA_AFFIL_COL,
        VT_AFFILIATION_NAMES, CS_FIELD_NAMES
    )
    print("Loaded configuration from validation:")
    print(f"  VT Affiliations: {VT_AFFILIATION_NAMES}")
    print(f"  CS Fields: {len(CS_FIELD_NAMES)} fields")
except ImportError:
    print("ERROR: validation_config.py not found")
    print("Run 01_validation.ipynb first")
    raise

# Processing configuration
CHUNK_SIZE = 100000

=== SciSciNet Preprocessing Notebook ===
Python version: 3.13.2 | packaged by Anaconda, Inc. | (main, Feb  6 2025, 19:03:27) [GCC 11.2.0]
Pandas version: 2.3.0

Loaded configuration from validation:
  VT Affiliations: ['Virginia Tech']
  CS Fields: 39 fields


In [2]:
print("\n=== Detecting Dataset Year Range ===\n")

# Detect the actual cutoff year from the dataset
papers_file = os.path.join(DATA_DIR, 'SciSciNet_Papers.tsv')

print("Scanning Papers.tsv to detect year range...")
print("(Scanning to find maximum year...)\n")

year_col = None
max_year = None
min_year = None
total_rows_scanned = 0
stable_count = 0  # Count how many chunks have the same max year

# Scan through chunks to find the maximum year
# Stop when max year is stable for 50 chunks (5M rows)
for chunk in pd.read_csv(papers_file, sep='\t', chunksize=CHUNK_SIZE):
    # Identify year column on first chunk
    if year_col is None:
        for col in ['Year', 'PublicationYear']:
            if col in chunk.columns:
                year_col = col
                break
        if year_col is None:
            raise ValueError("Could not find year column in Papers.tsv")
        print(f"Found year column: '{year_col}'")
    
    # Filter out NaN years
    valid_years = chunk[year_col].dropna()
    if len(valid_years) > 0:
        chunk_max = int(valid_years.max())
        chunk_min = int(valid_years.min())
        
        if max_year is None or chunk_max > max_year:
            max_year = chunk_max
            stable_count = 0  # Reset stability counter
        elif chunk_max == max_year:
            stable_count += 1
        else:
            stable_count = 0  # Reset if we see a lower max
        
        if min_year is None or chunk_min < min_year:
            min_year = chunk_min
    
    total_rows_scanned += len(chunk)
    
    # Print progress every 50 chunks
    if (total_rows_scanned // CHUNK_SIZE) % 50 == 0:
        print(f"  Scanned {total_rows_scanned:,} rows... Current max year: {max_year}")
    
    # Stop if max year has been stable for 50 chunks (5M rows)
    if stable_count >= 50:
        print(f"\nMax year stable at {max_year} for 50 chunks. Stopping scan.")
        break

print(f"\nCompleted scan: {total_rows_scanned:,} rows")

if max_year is None:
    raise ValueError("Could not detect year range from dataset")

# Calculate 10-year range from cutoff
CUTOFF_YEAR = int(max_year)
START_YEAR_T2 = CUTOFF_YEAR - 9  # 10 years inclusive (e.g., 2013-2022)
START_YEAR_T1 = CUTOFF_YEAR - 4   # 5 years inclusive (e.g., 2018-2022)
END_YEAR = CUTOFF_YEAR

# Database name with year range indicator - save to data directory
# DATA_DIR is '../data' relative to preprocessing/, so use it directly
DB_PATH = os.path.join(DATA_DIR, f'sciscinet_vt_cs_{START_YEAR_T2}_{END_YEAR}.db')

print(f"\nDataset year range detected:")
print(f"  Minimum year in dataset: {min_year}")
print(f"  Maximum year (cutoff): {CUTOFF_YEAR}")
print(f"\nCalculated 10-year range: {START_YEAR_T2}-{END_YEAR}")
print(f"\nProcessing Configuration:")
print(f"  Database: {DB_PATH}")
print(f"  Chunk size: {CHUNK_SIZE:,} rows")
print(f"\nYear Ranges:")
print(f"  Project 1 T1 (Citation & Collaboration): {START_YEAR_T1}-{END_YEAR} (past 5 years)")
print(f"  Project 1 T2 (Timeline): {START_YEAR_T2}-{END_YEAR} (past 10 years)")
print(f"  Project 2 (LLM Agent Analysis): Uses all papers from {START_YEAR_T2}-{END_YEAR}")



=== Detecting Dataset Year Range ===

Scanning Papers.tsv to detect year range...
(Scanning to find maximum year...)

Found year column: 'Year'
  Scanned 5,000,000 rows... Current max year: 2022

Max year stable at 2022 for 50 chunks. Stopping scan.

Completed scan: 5,100,000 rows

Dataset year range detected:
  Minimum year in dataset: 1800
  Maximum year (cutoff): 2022

Calculated 10-year range: 2013-2022

Processing Configuration:
  Database: ../data/sciscinet_vt_cs_2013_2022.db
  Chunk size: 100,000 rows

Year Ranges:
  Project 1 T1 (Citation & Collaboration): 2018-2022 (past 5 years)
  Project 1 T2 (Timeline): 2013-2022 (past 10 years)
  Project 2 (LLM Agent Analysis): Uses all papers from 2013-2022


In [3]:
print("\n=== Creating Database Schema ===\n")

# Preserve old database name if it exists (check both old and new locations)
OLD_DB_NAME_OLD_LOC = 'sciscinet_vt_cs.db'  # Old location in preprocessing/
OLD_DB_NAME_NEW_LOC = os.path.join(DATA_DIR, 'sciscinet_vt_cs.db')  # New location in data/

if os.path.exists(DB_PATH):
    # Only remove if it's not the old database name
    if DB_PATH != OLD_DB_NAME_OLD_LOC and DB_PATH != OLD_DB_NAME_NEW_LOC:
        os.remove(DB_PATH)
        print(f"Removed existing database: {DB_PATH}")
    else:
        print(f"Preserving old database: {DB_PATH}")
        print(f"New database will be created with a different name based on year range.")
elif os.path.exists(OLD_DB_NAME_OLD_LOC):
    print(f"Old database '{OLD_DB_NAME_OLD_LOC}' exists and will be preserved.")
    print(f"New database '{DB_PATH}' will be created separately.")
elif os.path.exists(OLD_DB_NAME_NEW_LOC):
    print(f"Old database '{OLD_DB_NAME_NEW_LOC}' exists and will be preserved.")
    print(f"New database '{DB_PATH}' will be created separately.")

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE affiliations (
    affiliation_id INTEGER PRIMARY KEY,
    affiliation_name TEXT
)
''')
print("[OK] Created table: affiliations")

cursor.execute('''
CREATE TABLE fields (
    field_id INTEGER PRIMARY KEY,
    field_name TEXT
)
''')
print("[OK] Created table: fields")

cursor.execute('''
CREATE TABLE papers (
    paper_id INTEGER PRIMARY KEY,
    title TEXT,
    year INTEGER,
    citation_count INTEGER,
    reference_count INTEGER,
    patent_count INTEGER
)
''')
print("[OK] Created table: papers")

cursor.execute('''
CREATE TABLE paper_details (
    paper_id INTEGER PRIMARY KEY,
    abstract TEXT
)
''')
print("[OK] Created table: paper_details")

cursor.execute('''
CREATE TABLE paper_author_affiliations (
    paper_id INTEGER,
    author_id INTEGER,
    affiliation_id INTEGER,
    author_sequence INTEGER,
    PRIMARY KEY (paper_id, author_id, affiliation_id)
)
''')
print("[OK] Created table: paper_author_affiliations")

cursor.execute('''
CREATE TABLE paper_fields (
    paper_id INTEGER,
    field_id INTEGER,
    score REAL,
    PRIMARY KEY (paper_id, field_id)
)
''')
print("[OK] Created table: paper_fields")

cursor.execute('''
CREATE TABLE paper_references (
    paper_id INTEGER,
    reference_id INTEGER,
    PRIMARY KEY (paper_id, reference_id)
)
''')
print("[OK] Created table: paper_references")

conn.commit()
print("\nDatabase schema created")


=== Creating Database Schema ===

Old database '../data/sciscinet_vt_cs.db' exists and will be preserved.
New database '../data/sciscinet_vt_cs_2013_2022.db' will be created separately.
[OK] Created table: affiliations
[OK] Created table: fields
[OK] Created table: papers
[OK] Created table: paper_details
[OK] Created table: paper_author_affiliations
[OK] Created table: paper_fields
[OK] Created table: paper_references

Database schema created


In [4]:
print("\n=== Loading Affiliations ===\n")

df_affiliations = pd.read_csv(
    os.path.join(DATA_DIR, 'SciSciNet_Affiliations.tsv'),
    sep='\t'
)

print(f"Total affiliations: {len(df_affiliations):,}")

vt_affiliations = df_affiliations[
    df_affiliations[AFFILIATION_ID_COL].isin(VT_AFFILIATION_IDS)
]

print(f"Virginia Tech affiliations: {len(vt_affiliations)}")

vt_affiliations_clean = vt_affiliations.rename(columns={
    AFFILIATION_ID_COL: 'affiliation_id',
    AFFILIATION_NAME_COL: 'affiliation_name'
})[['affiliation_id', 'affiliation_name']]

vt_affiliations_clean.to_sql('affiliations', conn, if_exists='replace', index=False)

print("Affiliations loaded to database")
print(f"\n{vt_affiliations_clean.to_string(index=False)}")


=== Loading Affiliations ===

Total affiliations: 26,998
Virginia Tech affiliations: 1
Affiliations loaded to database

 affiliation_id affiliation_name
      859038795    Virginia Tech


In [5]:
print("\n=== Loading Fields ===\n")

df_fields = pd.read_csv(
    os.path.join(DATA_DIR, 'SciSciNet_Fields.tsv'),
    sep='\t'
)

print(f"Total fields: {len(df_fields):,}")

cs_fields = df_fields[
    df_fields[FIELD_ID_COL].isin(CS_FIELD_IDS)
]

print(f"Computer Science fields: {len(cs_fields)}")

cs_fields_clean = cs_fields.rename(columns={
    FIELD_ID_COL: 'field_id',
    FIELD_NAME_COL: 'field_name'
})[['field_id', 'field_name']]

cs_fields_clean.to_sql('fields', conn, if_exists='replace', index=False)

print("Fields loaded to database")
print(f"\n{cs_fields_clean.to_string(index=False)}")


=== Loading Fields ===

Total fields: 311
Computer Science fields: 39
Fields loaded to database

  field_id                   field_name
 113775141         Computer engineering
 124101348                  Data mining
  56739046         Knowledge management
 149635348              Embedded system
 107457646   Human–computer interaction
  11413529                    Algorithm
  28490314           Speech recognition
 111919701             Operating system
  31972630              Computer vision
  77088390                     Database
 108827166             Internet privacy
  76155785           Telecommunications
   9390403            Computer hardware
 154945302      Artificial intelligence
  49774154                   Multimedia
    459310        Computational science
 199360897         Programming language
  38652104            Computer security
  79403827          Real-time computing
  41008148             Computer science
 204321447  Natural language processing
 188147891            

In [6]:
print("\n=== Step 1: Filtering Paper-Author-Affiliations for VT ===\n")
print("Processing large file (~11.68 GB)...\n")

file_path = os.path.join(DATA_DIR, 'SciSciNet_PaperAuthorAffiliations.tsv')
vt_papers_set = set()

chunk_count = 0
total_rows = 0
filtered_rows = 0
start_time = datetime.now()

for chunk in pd.read_csv(file_path, sep='\t', chunksize=CHUNK_SIZE):
    chunk_count += 1
    total_rows += len(chunk)
    
    vt_chunk = chunk[chunk[PAA_AFFIL_COL].isin(VT_AFFILIATION_IDS)]
    filtered_rows += len(vt_chunk)
    
    if len(vt_chunk) > 0:
        vt_papers_set.update(vt_chunk[PAPER_ID_COL].dropna().astype(int).values)
        
        # Determine sequence column
        seq_col = None
        for col in ['AuthorSequenceNumber', 'SequenceNumber', 'AuthorSequence']:
            if col in vt_chunk.columns:
                seq_col = col
                break
        
        cols_map = {
            PAPER_ID_COL: 'paper_id',
            AUTHOR_ID_COL: 'author_id',
            PAA_AFFIL_COL: 'affiliation_id'
        }
        if seq_col:
            cols_map[seq_col] = 'author_sequence'
        
        vt_chunk_clean = vt_chunk.rename(columns=cols_map)
        keep_cols = [col for col in ['paper_id', 'author_id', 'affiliation_id', 'author_sequence'] if col in vt_chunk_clean.columns]
        vt_chunk_clean[keep_cols].to_sql('paper_author_affiliations', conn, if_exists='append', index=False)
    
    if chunk_count % 50 == 0:
        elapsed = (datetime.now() - start_time).total_seconds()
        rate = total_rows / elapsed if elapsed > 0 else 0
        print(f"  Chunk {chunk_count}: {total_rows:,} rows | {filtered_rows:,} VT records | {rate:.0f} rows/sec")

elapsed_time = (datetime.now() - start_time).total_seconds()
print(f"\nStep 1 Complete")
print(f"  Time: {elapsed_time:.1f} seconds")
print(f"  Total rows: {total_rows:,}")
print(f"  VT records: {filtered_rows:,}")
print(f"  Unique VT papers: {len(vt_papers_set):,}")

conn.commit()


=== Step 1: Filtering Paper-Author-Affiliations for VT ===

Processing large file (~11.68 GB)...

  Chunk 50: 5,000,000 rows | 1,933 VT records | 2126851 rows/sec
  Chunk 100: 10,000,000 rows | 3,922 VT records | 2118229 rows/sec
  Chunk 150: 15,000,000 rows | 4,954 VT records | 2177680 rows/sec
  Chunk 200: 20,000,000 rows | 5,742 VT records | 2227797 rows/sec
  Chunk 250: 25,000,000 rows | 7,701 VT records | 2182043 rows/sec
  Chunk 300: 30,000,000 rows | 9,959 VT records | 2145627 rows/sec
  Chunk 350: 35,000,000 rows | 12,298 VT records | 2125553 rows/sec
  Chunk 400: 40,000,000 rows | 14,323 VT records | 2106511 rows/sec
  Chunk 450: 45,000,000 rows | 17,179 VT records | 2084872 rows/sec
  Chunk 500: 50,000,000 rows | 20,514 VT records | 2058679 rows/sec
  Chunk 550: 55,000,000 rows | 23,959 VT records | 961225 rows/sec
  Chunk 600: 60,000,000 rows | 27,480 VT records | 1002611 rows/sec
  Chunk 650: 65,000,000 rows | 30,837 VT records | 1040894 rows/sec
  Chunk 700: 70,000,000 ro

In [7]:
print("\n=== Step 2: Filtering Papers for VT + Year Range ===\n")
print("Processing Papers.tsv (~16.46 GB)...\n")
print(f"Using year range: {START_YEAR_T2}-{END_YEAR} (to support all projects)\n")

file_path = os.path.join(DATA_DIR, 'SciSciNet_Papers.tsv')
vt_papers_in_range_set = set()

chunk_count = 0
total_rows = 0
filtered_rows = 0
start_time = datetime.now()

for chunk in pd.read_csv(file_path, sep='\t', chunksize=CHUNK_SIZE):
    chunk_count += 1
    total_rows += len(chunk)
    
    # Identify year column
    year_col = None
    for col in ['Year', 'PublicationYear']:
        if col in chunk.columns:
            year_col = col
            break
    
    filtered_chunk = chunk[
        (chunk[PAPER_ID_COL].isin(vt_papers_set)) &
        (chunk[year_col] >= START_YEAR_T2) &
        (chunk[year_col] <= END_YEAR)
    ]
    filtered_rows += len(filtered_chunk)
    
    if len(filtered_chunk) > 0:
        vt_papers_in_range_set.update(filtered_chunk[PAPER_ID_COL].dropna().astype(int).values)
        
        # Identify columns
        title_col = None
        for col in ['PaperTitle', 'OriginalTitle', 'Title']:
            if col in filtered_chunk.columns:
                title_col = col
                break
        
        citation_col = None
        for col in ['CitationCount', 'Citations', 'Citation_Count']:
            if col in filtered_chunk.columns:
                citation_col = col
                break
        
        reference_col = None
        for col in ['ReferenceCount', 'References', 'Reference_Count']:
            if col in filtered_chunk.columns:
                reference_col = col
                break
        
        patent_col = None
        for col in ['Patent_Count', 'PatentCount']:
            if col in filtered_chunk.columns:
                patent_col = col
                break
        
        cols_map = {PAPER_ID_COL: 'paper_id'}
        if year_col:
            cols_map[year_col] = 'year'
        if title_col:
            cols_map[title_col] = 'title'
        if citation_col:
            cols_map[citation_col] = 'citation_count'
        if reference_col:
            cols_map[reference_col] = 'reference_count'
        if patent_col:
            cols_map[patent_col] = 'patent_count'
        
        filtered_chunk_clean = filtered_chunk.rename(columns=cols_map)
        
        keep_cols = [col for col in ['paper_id', 'title', 'year', 'citation_count', 'reference_count', 'patent_count'] if col in filtered_chunk_clean.columns]
        
        filtered_chunk_clean[keep_cols].to_sql('papers', conn, if_exists='append', index=False)
    
    if chunk_count % 50 == 0:
        elapsed = (datetime.now() - start_time).total_seconds()
        rate = total_rows / elapsed if elapsed > 0 else 0
        print(f"  Chunk {chunk_count}: {total_rows:,} rows | {filtered_rows:,} filtered | {rate:.0f} rows/sec")

elapsed_time = (datetime.now() - start_time).total_seconds()
print(f"\nStep 2 Complete")
print(f"  Time: {elapsed_time:.1f} seconds")
print(f"  VT papers in year range: {len(vt_papers_in_range_set):,}")

conn.commit()


=== Step 2: Filtering Papers for VT + Year Range ===

Processing Papers.tsv (~16.46 GB)...

Using year range: 2013-2022 (to support all projects)

  Chunk 50: 5,000,000 rows | 1,480 filtered | 278465 rows/sec
  Chunk 100: 10,000,000 rows | 2,957 filtered | 278331 rows/sec
  Chunk 150: 15,000,000 rows | 4,421 filtered | 277349 rows/sec
  Chunk 200: 20,000,000 rows | 5,910 filtered | 276762 rows/sec
  Chunk 250: 25,000,000 rows | 7,358 filtered | 276964 rows/sec
  Chunk 300: 30,000,000 rows | 8,824 filtered | 276984 rows/sec
  Chunk 350: 35,000,000 rows | 10,315 filtered | 276972 rows/sec
  Chunk 400: 40,000,000 rows | 11,782 filtered | 276755 rows/sec
  Chunk 450: 45,000,000 rows | 13,255 filtered | 276576 rows/sec
  Chunk 500: 50,000,000 rows | 14,822 filtered | 276250 rows/sec
  Chunk 550: 55,000,000 rows | 16,309 filtered | 275855 rows/sec
  Chunk 600: 60,000,000 rows | 17,809 filtered | 275579 rows/sec
  Chunk 650: 65,000,000 rows | 19,291 filtered | 275073 rows/sec
  Chunk 700: 70

In [8]:
print("\n=== Step 3: Filtering for Computer Science Papers ===\n")
print("Processing PaperFields.tsv (~11.62 GB)...\n")

file_path = os.path.join(DATA_DIR, 'SciSciNet_PaperFields.tsv')
final_cs_papers_set = set()

chunk_count = 0
total_rows = 0
filtered_rows = 0
start_time = datetime.now()

# Peek at columns
sample = pd.read_csv(file_path, sep='\t', nrows=1)
pf_paper_col = None
pf_field_col = None
pf_score_col = None

for col in sample.columns:
    col_lower = col.lower()
    if 'paper' in col_lower and 'id' in col_lower:
        pf_paper_col = col
    if 'field' in col_lower and 'id' in col_lower:
        pf_field_col = col
    if 'score' in col_lower:
        pf_score_col = col

print(f"Using columns: paper='{pf_paper_col}', field='{pf_field_col}', score='{pf_score_col}'\n")

for chunk in pd.read_csv(file_path, sep='\t', chunksize=CHUNK_SIZE):
    chunk_count += 1
    total_rows += len(chunk)
    
    cs_chunk = chunk[
        (chunk[pf_paper_col].isin(vt_papers_in_range_set)) &
        (chunk[pf_field_col].isin(CS_FIELD_IDS))
    ]
    filtered_rows += len(cs_chunk)
    
    if len(cs_chunk) > 0:
        final_cs_papers_set.update(cs_chunk[pf_paper_col].dropna().astype(int).values)
        
        cols_map = {
            pf_paper_col: 'paper_id',
            pf_field_col: 'field_id'
        }
        if pf_score_col:
            cols_map[pf_score_col] = 'score'
        
        cs_chunk_clean = cs_chunk.rename(columns=cols_map)
        keep_cols = [col for col in ['paper_id', 'field_id', 'score'] if col in cs_chunk_clean.columns]
        cs_chunk_clean[keep_cols].to_sql('paper_fields', conn, if_exists='append', index=False)
    
    if chunk_count % 50 == 0:
        elapsed = (datetime.now() - start_time).total_seconds()
        rate = total_rows / elapsed if elapsed > 0 else 0
        print(f"  Chunk {chunk_count}: {total_rows:,} rows | {filtered_rows:,} CS papers | {rate:.0f} rows/sec")

elapsed_time = (datetime.now() - start_time).total_seconds()
print(f"\nStep 3 Complete")
print(f"  Time: {elapsed_time:.1f} seconds")
print(f"  Final VT CS papers: {len(final_cs_papers_set):,}")

conn.commit()


=== Step 3: Filtering for Computer Science Papers ===

Processing PaperFields.tsv (~11.62 GB)...

Using columns: paper='PaperID', field='FieldID', score='None'

  Chunk 50: 5,000,000 rows | 88 CS papers | 1199755 rows/sec
  Chunk 100: 10,000,000 rows | 203 CS papers | 1194513 rows/sec
  Chunk 150: 15,000,000 rows | 376 CS papers | 1182288 rows/sec
  Chunk 200: 20,000,000 rows | 503 CS papers | 1175363 rows/sec
  Chunk 250: 25,000,000 rows | 620 CS papers | 1171805 rows/sec
  Chunk 300: 30,000,000 rows | 751 CS papers | 1170495 rows/sec
  Chunk 350: 35,000,000 rows | 881 CS papers | 1169634 rows/sec
  Chunk 400: 40,000,000 rows | 1,028 CS papers | 1169598 rows/sec
  Chunk 450: 45,000,000 rows | 1,174 CS papers | 1169503 rows/sec
  Chunk 500: 50,000,000 rows | 1,305 CS papers | 1168932 rows/sec
  Chunk 550: 55,000,000 rows | 1,445 CS papers | 1168822 rows/sec
  Chunk 600: 60,000,000 rows | 1,714 CS papers | 1171321 rows/sec
  Chunk 650: 65,000,000 rows | 1,843 CS papers | 1177072 rows/s

In [9]:
print("\n=== Step 4: Building Citation Network ===\n")
print("Processing PaperReferences.tsv (~32.41 GB)...\n")

file_path = os.path.join(DATA_DIR, 'SciSciNet_PaperReferences.tsv')

chunk_count = 0
total_rows = 0
filtered_rows = 0
start_time = datetime.now()

# Peek at columns
sample = pd.read_csv(file_path, sep='\t', nrows=1)
print(f"Available columns: {sample.columns.tolist()}\n")

pr_paper_col = None
pr_ref_col = None

# Try to identify the paper column (citing paper)
for col in sample.columns:
    col_lower = col.lower().replace(' ', '').replace('_', '')
    # Look for main paper ID column (the one doing the citing)
    if col_lower in ['paperid', 'citingpaperid']:
        pr_paper_col = col
        break
    if 'paper' in col_lower and 'id' in col_lower and 'cited' not in col_lower and 'reference' not in col_lower:
        pr_paper_col = col

# Try to identify the reference column (cited paper)
for col in sample.columns:
    col_lower = col.lower().replace(' ', '').replace('_', '')
    # Look for cited/reference paper ID column
    if col_lower in ['referenceid', 'citedpaperid', 'referencepaperid']:
        pr_ref_col = col
        break
    if ('cited' in col_lower or 'reference' in col_lower) and 'id' in col_lower:
        pr_ref_col = col

print(f"Detected columns:")
print(f"  Paper (citing): '{pr_paper_col}'")
print(f"  Reference (cited): '{pr_ref_col}'\n")

# Validate that both columns were found
if pr_paper_col is None or pr_ref_col is None:
    raise ValueError(
        f"Could not identify paper or reference columns!\n"
        f"Available columns: {sample.columns.tolist()}\n"
        f"Please manually specify the correct column names."
    )

# Process the file in chunks
for chunk in pd.read_csv(file_path, sep='\t', chunksize=CHUNK_SIZE):
    chunk_count += 1
    total_rows += len(chunk)
    
    # Filter for citations involving our CS papers
    # Include citations where either the citing paper OR cited paper is in our set
    filtered_chunk = chunk[
        (chunk[pr_paper_col].isin(final_cs_papers_set)) |
        (chunk[pr_ref_col].isin(final_cs_papers_set))
    ]
    filtered_rows += len(filtered_chunk)
    
    if len(filtered_chunk) > 0:
        filtered_chunk_clean = filtered_chunk.rename(columns={
            pr_paper_col: 'paper_id',
            pr_ref_col: 'reference_id'
        })[['paper_id', 'reference_id']]
        
        filtered_chunk_clean.to_sql('paper_references', conn, if_exists='append', index=False)
    
    if chunk_count % 50 == 0:
        elapsed = (datetime.now() - start_time).total_seconds()
        rate = total_rows / elapsed if elapsed > 0 else 0
        print(f"  Chunk {chunk_count}: {total_rows:,} rows | {filtered_rows:,} citations | {rate:.0f} rows/sec")

elapsed_time = (datetime.now() - start_time).total_seconds()
print(f"\nStep 4 Complete")
print(f"  Time: {elapsed_time:.1f} seconds")
print(f"  Citation links: {filtered_rows:,}")

conn.commit()


=== Step 4: Building Citation Network ===

Processing PaperReferences.tsv (~32.41 GB)...

Available columns: ['Citing_PaperID', 'Cited_PaperID']

Detected columns:
  Paper (citing): 'Citing_PaperID'
  Reference (cited): 'Cited_PaperID'

  Chunk 50: 5,000,000 rows | 134 citations | 2721945 rows/sec
  Chunk 100: 10,000,000 rows | 470 citations | 2786990 rows/sec
  Chunk 150: 15,000,000 rows | 656 citations | 2777544 rows/sec
  Chunk 200: 20,000,000 rows | 1,060 citations | 2789385 rows/sec
  Chunk 250: 25,000,000 rows | 1,167 citations | 2776417 rows/sec
  Chunk 300: 30,000,000 rows | 1,380 citations | 2763943 rows/sec
  Chunk 350: 35,000,000 rows | 1,565 citations | 2750236 rows/sec
  Chunk 400: 40,000,000 rows | 1,985 citations | 2731276 rows/sec
  Chunk 450: 45,000,000 rows | 2,386 citations | 2722707 rows/sec
  Chunk 500: 50,000,000 rows | 3,069 citations | 2711202 rows/sec
  Chunk 550: 55,000,000 rows | 3,781 citations | 2703653 rows/sec
  Chunk 600: 60,000,000 rows | 4,626 citatio

In [10]:
print("\n=== Step 5: Adding Paper Details ===\n")
print("Processing PaperDetails.tsv (~28.72 GB)...\n")

file_path = os.path.join(DATA_DIR, 'SciSciNet_PaperDetails.tsv')

chunk_count = 0
total_rows = 0
filtered_rows = 0
start_time = datetime.now()

# Peek at columns
sample = pd.read_csv(file_path, sep='\t', nrows=1)
available_cols = sample.columns.tolist()
print(f"Available columns: {available_cols}\n")

usecols = [PAPER_ID_COL]
abstract_col = None

for col in ['Abstract', 'OriginalAbstract']:
    if col in available_cols:
        abstract_col = col
        usecols.append(col)
        break


print(f"Extracting columns: {usecols}\n")

for chunk in pd.read_csv(file_path, sep='\t', chunksize=CHUNK_SIZE, usecols=usecols):
    chunk_count += 1
    total_rows += len(chunk)
    
    filtered_chunk = chunk[chunk[PAPER_ID_COL].isin(final_cs_papers_set)]
    filtered_rows += len(filtered_chunk)
    
    if len(filtered_chunk) > 0:
        cols_map = {PAPER_ID_COL: 'paper_id'}
        if abstract_col:
            cols_map[abstract_col] = 'abstract'
        
        filtered_chunk_clean = filtered_chunk.rename(columns=cols_map)
        filtered_chunk_clean.to_sql('paper_details', conn, if_exists='append', index=False)
    
    if chunk_count % 30 == 0:
        elapsed = (datetime.now() - start_time).total_seconds()
        rate = total_rows / elapsed if elapsed > 0 else 0
        print(f"  Chunk {chunk_count}: {total_rows:,} rows | {filtered_rows:,} details | {rate:.0f} rows/sec")

elapsed_time = (datetime.now() - start_time).total_seconds()
print(f"\nStep 5 Complete")
print(f"  Time: {elapsed_time:.1f} seconds")
print(f"  Paper details added: {filtered_rows:,}")

conn.commit()


=== Step 5: Adding Paper Details ===

Processing PaperDetails.tsv (~28.72 GB)...

Available columns: ['PaperID', 'DOI', 'DocType', 'PaperTitle', 'BookTitle', 'Year', 'Date', 'Publisher', 'JournalID', 'ConferenceSeriesID', 'Volume', 'Issue', 'FirstPage', 'LastPage', 'ReferenceCount', 'CitationCount', 'OriginalVenue', 'FamilyID', 'RetractionType']

Extracting columns: ['PaperID']

  Chunk 30: 3,000,000 rows | 220 details | 401912 rows/sec
  Chunk 60: 6,000,000 rows | 468 details | 401643 rows/sec
  Chunk 90: 9,000,000 rows | 701 details | 401145 rows/sec
  Chunk 120: 12,000,000 rows | 908 details | 401309 rows/sec
  Chunk 150: 15,000,000 rows | 1,133 details | 401430 rows/sec
  Chunk 180: 18,000,000 rows | 1,350 details | 401276 rows/sec
  Chunk 210: 21,000,000 rows | 1,572 details | 254383 rows/sec
  Chunk 240: 24,000,000 rows | 1,795 details | 266589 rows/sec
  Chunk 270: 27,000,000 rows | 2,023 details | 276889 rows/sec
  Chunk 300: 30,000,000 rows | 2,261 details | 285688 rows/sec
 

In [11]:
print("\n=== Creating Database Indexes ===\n")

indexes = [
    ("idx_papers_year", "CREATE INDEX idx_papers_year ON papers(year)"),
    ("idx_papers_id", "CREATE INDEX idx_papers_id ON papers(paper_id)"),
    ("idx_paa_paper", "CREATE INDEX idx_paa_paper ON paper_author_affiliations(paper_id)"),
    ("idx_paa_author", "CREATE INDEX idx_paa_author ON paper_author_affiliations(author_id)"),
    ("idx_paa_affil", "CREATE INDEX idx_paa_affil ON paper_author_affiliations(affiliation_id)"),
    ("idx_pf_paper", "CREATE INDEX idx_pf_paper ON paper_fields(paper_id)"),
    ("idx_pf_field", "CREATE INDEX idx_pf_field ON paper_fields(field_id)"),
    ("idx_pr_paper", "CREATE INDEX idx_pr_paper ON paper_references(paper_id)"),
    ("idx_pr_ref", "CREATE INDEX idx_pr_ref ON paper_references(reference_id)"),
    ("idx_pd_paper", "CREATE INDEX idx_pd_paper ON paper_details(paper_id)")
]

for idx_name, idx_sql in indexes:
    try:
        cursor.execute(idx_sql)
        print(f"[OK] {idx_name}")
    except Exception as e:
        print(f"[ERROR] {idx_name}: {e}")

conn.commit()
print("\nIndexes created")


=== Creating Database Indexes ===

[OK] idx_papers_year
[OK] idx_papers_id
[OK] idx_paa_paper
[OK] idx_paa_author
[OK] idx_paa_affil
[OK] idx_pf_paper
[OK] idx_pf_field
[OK] idx_pr_paper
[OK] idx_pr_ref
[OK] idx_pd_paper

Indexes created


In [12]:
print("\n" + "="*120)
print("DATABASE SUMMARY")
print("="*120 + "\n")

tables = [
    ('affiliations', 'Affiliations'),
    ('fields', 'Fields'),
    ('papers', 'Papers'),
    ('paper_details', 'Paper Details'),
    ('paper_author_affiliations', 'Paper-Author-Affiliations'),
    ('paper_fields', 'Paper-Fields'),
    ('paper_references', 'Citations')
]

print("Row Counts:\n")
for table, display_name in tables:
    count = pd.read_sql(f"SELECT COUNT(*) as count FROM {table}", conn).iloc[0]['count']
    print(f"  {display_name:35} {count:>12,} rows")

print("\nPapers by Year:\n")
papers_by_year = pd.read_sql("""
    SELECT year, COUNT(*) as count 
    FROM papers 
    GROUP BY year 
    ORDER BY year
""", conn)
print(papers_by_year.to_string(index=False))

print("\nCitation Network:\n")
citation_stats = pd.read_sql("""
    SELECT 
        COUNT(*) as total_citation_links,
        COUNT(DISTINCT paper_id) as papers_that_cite,
        COUNT(DISTINCT reference_id) as papers_cited
    FROM paper_references
""", conn)
print(citation_stats.to_string(index=False))

print("\nAuthor Statistics:\n")
author_stats = pd.read_sql("""
    SELECT 
        COUNT(DISTINCT author_id) as unique_authors,
        COUNT(DISTINCT paper_id) as papers_with_authors,
        ROUND(CAST(COUNT(*) AS FLOAT) / COUNT(DISTINCT paper_id), 2) as avg_authors_per_paper
    FROM paper_author_affiliations
""", conn)
print(author_stats.to_string(index=False))

print("\nPatent Statistics:\n")
patent_stats = pd.read_sql("""
    SELECT 
        COUNT(patent_count) as papers_with_patent_data,
        SUM(CASE WHEN patent_count > 0 THEN 1 ELSE 0 END) as papers_with_patents,
        ROUND(AVG(patent_count), 2) as avg_patent_count,
        MAX(patent_count) as max_patent_count
    FROM papers
    WHERE patent_count IS NOT NULL
""", conn)
print(patent_stats.to_string(index=False))

db_size_mb = os.path.getsize(DB_PATH) / (1024**2)
print(f"\nDatabase File:")
print(f"  Location: {os.path.abspath(DB_PATH)}")
print(f"  Size: {db_size_mb:.2f} MB")

conn.close()

print("\n" + "="*120)
print("PREPROCESSING COMPLETE")
print("="*120)
print(f"\nDatabase ready for both projects:")
print(f"\nProject 1 - Full-Stack Web Development:")
print(f"  T1 (Citation & Collaboration Networks): {START_YEAR_T1}-{END_YEAR} (past 5 years)")
print(f"  T2 (Timeline + Patent Distribution): {START_YEAR_T2}-{END_YEAR} (past 10 years)")
print(f"  T3 (Network Refinement): Uses T1 data")
print(f"\nProject 2 - LLM Agent Analysis:")
print(f"  Uses all VT CS papers from {START_YEAR_T2}-{END_YEAR}")
print(f"  All tables filtered for VT + CS + year range")
print(f"\nDatabase location: {os.path.abspath(DB_PATH)}")
print("="*120)


DATABASE SUMMARY

Row Counts:

  Affiliations                                   1 rows
  Fields                                        39 rows
  Papers                                    39,903 rows
  Paper Details                             10,293 rows
  Paper-Author-Affiliations                193,408 rows
  Paper-Fields                              18,047 rows
  Citations                                424,616 rows

Papers by Year:

 year  count
 2013   3453
 2014   3776
 2015   4190
 2016   4193
 2017   4495
 2018   4439
 2019   4845
 2020   5096
 2021   5307
 2022    109

Citation Network:

 total_citation_links  papers_that_cite  papers_cited
               424616            125892        199122

Author Statistics:

 unique_authors  papers_with_authors  avg_authors_per_paper
          42152                94577                   2.04

Patent Statistics:

 papers_with_patent_data  papers_with_patents  avg_patent_count  max_patent_count
                   39903                 10

In [13]:
print("\n=== Year Range Verification ===\n")

conn = sqlite3.connect(DB_PATH)

print("Papers by year with applicable tasks:\n")
year_summary = pd.read_sql(f"""
    SELECT 
        year,
        COUNT(*) as total_papers,
        CASE 
            WHEN year >= {START_YEAR_T1} THEN 'Project 1 (T1, T2, T3) + Project 2'
            WHEN year >= {START_YEAR_T2} THEN 'Project 1 (T2) + Project 2'
            ELSE 'Out of range'
        END as applicable_to
    FROM papers 
    GROUP BY year 
    ORDER BY year DESC
""", conn)
print(year_summary.to_string(index=False))

t1_count = pd.read_sql(f"""
    SELECT COUNT(*) as count FROM papers WHERE year >= {START_YEAR_T1}
""", conn).iloc[0]['count']

t2_count = pd.read_sql(f"""
    SELECT COUNT(*) as count FROM papers WHERE year >= {START_YEAR_T2}
""", conn).iloc[0]['count']

print(f"\n\nPapers available for:")
print(f"  Project 1 T1 (Citation & Collaboration, {START_YEAR_T1}-{END_YEAR}): {t1_count:,} papers")
print(f"  Project 1 T2 (Timeline, {START_YEAR_T2}-{END_YEAR}): {t2_count:,} papers")
print(f"  Project 2 (LLM Agent, {START_YEAR_T2}-{END_YEAR}): {t2_count:,} papers")

conn.close()


=== Year Range Verification ===

Papers by year with applicable tasks:

 year  total_papers                      applicable_to
 2022           109 Project 1 (T1, T2, T3) + Project 2
 2021          5307 Project 1 (T1, T2, T3) + Project 2
 2020          5096 Project 1 (T1, T2, T3) + Project 2
 2019          4845 Project 1 (T1, T2, T3) + Project 2
 2018          4439 Project 1 (T1, T2, T3) + Project 2
 2017          4495         Project 1 (T2) + Project 2
 2016          4193         Project 1 (T2) + Project 2
 2015          4190         Project 1 (T2) + Project 2
 2014          3776         Project 1 (T2) + Project 2
 2013          3453         Project 1 (T2) + Project 2


Papers available for:
  Project 1 T1 (Citation & Collaboration, 2018-2022): 19,796 papers
  Project 1 T2 (Timeline, 2013-2022): 39,903 papers
  Project 2 (LLM Agent, 2013-2022): 39,903 papers


In [14]:
print("\n=== Sample Queries ===\n")

conn = sqlite3.connect(DB_PATH)

print("Most recent 5 papers:\n")
recent_papers = pd.read_sql("""
    SELECT paper_id, title, year, citation_count
    FROM papers
    ORDER BY year DESC, citation_count DESC
    LIMIT 5
""", conn)
print(recent_papers.to_string(index=False))

print("\n\nTop 5 most cited papers:\n")
top_cited = pd.read_sql("""
    SELECT paper_id, title, year, citation_count
    FROM papers
    WHERE citation_count IS NOT NULL
    ORDER BY citation_count DESC
    LIMIT 5
""", conn)
print(top_cited.to_string(index=False))

print("\n\nPapers with most patent citations:\n")
top_patents = pd.read_sql("""
    SELECT p.paper_id, p.title, p.year, p.patent_count
    FROM papers p
    WHERE p.patent_count > 0
    ORDER BY p.patent_count DESC
    LIMIT 5
""", conn)
print(top_patents.to_string(index=False))

print("\n\nMost prolific authors:\n")
top_authors = pd.read_sql("""
    SELECT author_id, COUNT(DISTINCT paper_id) as paper_count
    FROM paper_author_affiliations
    GROUP BY author_id
    ORDER BY paper_count DESC
    LIMIT 5
""", conn)
print(top_authors.to_string(index=False))

print("\n\nPapers by field (for Project 2 analysis):\n")
papers_by_field = pd.read_sql("""
    SELECT f.field_name, COUNT(DISTINCT pf.paper_id) as paper_count
    FROM paper_fields pf
    JOIN fields f ON pf.field_id = f.field_id
    GROUP BY f.field_name
    ORDER BY paper_count DESC
""", conn)
print(papers_by_field.to_string(index=False))

conn.close()

print("\n\nSample queries complete")
print("Database ready for Project 1 and Project 2!")


=== Sample Queries ===

Most recent 5 papers:

  paper_id title  year  citation_count
3159659315  None  2022              13
3210404677  None  2022               3
3197283713  None  2022               2
3194526859  None  2022               2
3193947462  None  2022               2


Top 5 most cited papers:

  paper_id title  year  citation_count
2751884637  None  2017            8514
2104549677  None  2015            8179
2614986146  None  2016            3451
2527824850  None  2016            3429
2142428670  None  2014            2506


Papers with most patent citations:

  paper_id title  year  patent_count
2010121929  None  2014            34
1988386651  None  2013            28
2104549677  None  2015            24
1973838853  None  2013            23
2010869577  None  2014            22


Most prolific authors:

 author_id  paper_count
1750292016          481
1263832234          445
1865774345          405
1972755273          398
2102906916          374


Papers by field (for Pro